In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, \
    SeparableConv2D, GlobalAveragePooling2D
from keras.utils import to_categorical, Sequence
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import initializers
from numpy.random import seed
import json
import random
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob
import imgaug.augmenters as iaa
import imgaug as ia

ImportError: No module named 'imgaug'

In [ ]:
labels = "./dataset/labels_for_test.json"
with open(labels) as label:
    data = json.load(label) 

In [ ]:
def data_prepare(labels):
    data = {}
    for key in labels.keys():
        if labels[key] not in data:
            data.update({labels[key]: []})
        data[labels[key]].append(key)
    return data

In [ ]:
data  = data_prepare(data)

In [ ]:
max_len = 0
train_data = {}
val_data = {}
for i in data:
    if len(data[i]) > max_len:
        max_len = len(data[i])
    train_data.update({i: data[i][:int(0.9*len(data[i]))]})
    val_data.update({i: data[i][int(0.9*len(data[i])):]})

In [ ]:
img_width, img_height = 256, 256

seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    sometimes(iaa.Affine(
    rotate=(-45, 45),
)),
])
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

for label in train_data:
    if not os.path.exists('dataset/learning_data/' + label):
        os.makedirs('dataset/learning_data/' + label)
    for i, img_name in enumerate(train_data[label]):
        image = cv2.imread('dataset/data_for_test/' +  img_name)
        gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
        cv2.imwrite('dataset/learning_data/' + label + "/" + str(i) +".jpg", gray)
    for i in range(max_len-len(train_data[label])):
        image = cv2.imread('dataset/data_for_test/' + random.choice(train_data[label]))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        images_aug = seq(images=gray)
        cv2.imwrite('dataset/learning_data/' + label + "/" + str(len(train_data[label]) + i) +".jpg", images_aug)

for label in val_data:
    if not os.path.exists('dataset/validation_data/' + label):
        os.makedirs('dataset/validation_data/' + label)
    for i, img_name in enumerate(val_data[label]):
        image = cv2.imread('dataset/data_for_test/' +  img_name)
        gray = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), (img_width, img_height))
        cv2.imwrite('dataset/validation_data/' + label + "/" + str(i) +".jpg", gray)
        
del(val_data)
del(train_data)
del(data)

In [ ]:
train_data_dir = "dataset/learning_data/"
validation_data_dir = "dataset/validation_data/"
nb_train_samples = 20
nb_validation_samples = 2
epochs = 5
batch_size = 16
base_model = applications.VGG16(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu',
          kernel_initializer=initializers.RandomUniform(minval=-0.05, maxval=0.05, seed=None))(x)

predictions = Dense(8, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples)

In [ ]:
class Seq(Sequence):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]["image"]
        batch_y = self.data[idx * self.batch_size:(idx + 1) * self.batch_size]["label"]

        return (np.array([cv2.resize(cv2.imread(file_name, 1), (256, 256)) for file_name in batch_x]),
                keras.utils.to_categorical(np.array(batch_y), num_classes =8))

def build_model():
    input_img = Input(shape=(256,256,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.1, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.1, name='dropout2')(x)
    x = Dense(8, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model
model =  build_model()

model =  build_model()

opt = Adam(lr=0.0001, decay=1e-5)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

batch_size = 8
nb_epochs = 20

train_data_gen = Seq(data=train_data, batch_size=batch_size)

nb_train_steps = train_data.shape[0]//batch_size

history = model.fit_generator(generator=train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                               validation_data=(valid_data, valid_labels),callbacks=[es, chkpt])